In [1]:
import json
from pyairtable import Table
import pandas as pd
from fuzzywuzzy import fuzz

In [2]:
with open("../airtable_apikey.json") as f:  
        keys = json.load(f)

base_id = keys[0]['base_id']
api_key = keys[0]['api_key']

In [3]:
def process_airtable(airtable_output, retrive_var = []):

    temp_dict = {}

    for i in retrive_var:
        if i == "id":
            temp_dict[i] = airtable_output['id']
        else:
            if i in airtable_output['fields'].keys():
                temp_dict[i.lower()] = airtable_output['fields'][i]
            else:
                temp_dict[i.lower()] = ''

    return temp_dict

In [4]:
content_table_id = keys[0]['content_table_id']
content_table = Table(api_key, base_id, content_table_id).all()
content_list = []
for i in content_table:
    temp_dict = process_airtable(i, retrive_var=['id', 'Type','Title','Writer', 'Value Props', 'Appeals to (from Value Props)'])
    content_list.append(temp_dict)


In [5]:
value_props_id = keys[0]['value_props_id']
value_props = Table(api_key, base_id, value_props_id).all()

value_props_name = []

for i in value_props:
    temp_dict = process_airtable(i, retrive_var=['id', 'Name'])
    value_props_name.append(temp_dict)

In [6]:
persona_table_id = keys[0]['persona_table']
persona_table = Table(api_key, base_id, persona_table_id).all()

persona_list = []

for i in persona_table:
    temp_dict = process_airtable(i, retrive_var=['id', 'Persona'])
    persona_list.append(temp_dict)

In [7]:
for i in content_list:

    i['value props name'] = ''
    i['persona'] = ''

    if i['value props'] != '':
        for j in i['value props']:
            for m in value_props_name:
                if m['id'] == j:
                    i['value props name'] = i['value props name'] + m['name'] + ' '
                    
    i['value props name'] = i['value props name'].strip()

    if i['appeals to (from value props)'] != '':
        for j in i['appeals to (from value props)']:
            for m in persona_list:
                if m['id'] == j:
                    i['persona'] = i['persona'] + m['persona'] + ' '
                    
    i['persona'] = i['persona'].strip()

    if i['writer'] != '':
        i['writer name'] = i['writer']['name']
    else:
        i['writer name'] = ''

    del i['value props']
    del i['appeals to (from value props)']
    del i['writer']


In [8]:
content_df = pd.DataFrame(content_list)

In [9]:
content_df['lookup'] = content_df.apply(lambda x: x['type'] + ' ' + x.title + x['value props name'] + ' ' + x.persona + ' ' + x['writer name'], axis = 1)

In [37]:
fuzz.partial_ratio('christelle', 'christelle bron xbud')

100

In [11]:
import jellyfish

In [30]:
from autocorrect import Speller
spell = Speller(lang='en')

keyword = 'christelle'
keyword = spell(keyword)
keyword

'christelle'

In [19]:
content_df['score'] = 0

for index, row in content_df.iterrows():
    score = 0
    for col in ['type','title','value props name', 'persona', 'writer name']:
        if row[col] != '':

            score += jellyfish.levenshtein_distance(keyword.lower(), row[col].lower())

    content_df.loc[index, 'score'] = score


In [33]:
jellyfish.levenshtein_distance('christelle', 'chrstelle')

1

In [38]:
content_df.sort_values('score',ascending=True)